Que pasa si se ingresa una nueva pelicula o un conjunto de nuevas peliculas?

Este notebook esta dedicado a eso, al proceso de actualizar nuestra base de datos, realizar las transformaciones necesarias e implementarlas en nuestro modelo de recomendacion.

Esto es, la creacion de un PIPELINE. Una secuencia de transformaciones realizadas a los datos.

In [1]:
import pandas as pd
import numpy as np

import ast

import re
import spacy
# !python -m spacy download en_core_web_sm

from sklearn.preprocessing import RobustScaler

import time

## Cargamos los nuevos datos

In [2]:
# usaremos data previa para "recrear" el caso
data = pd.read_csv('../data/movies_dataset.csv', low_memory=False)

Se asume que los datos vienen en un formato de dato, esto es:

1. El nombre de las columnas
2. Los tipos de datos en cada columna

Por lo que previamente se le debe realizar una mini-transformacion en caso de que sea necesario.

## Pipeline

In [3]:
def pipeline(data):
    # Si se ejecuta cada paso correctamente, entonces retorna True
    
    # Primera etapa
    output = clean_data(data)
    
    # Segunda etapa
    output = feature_engineer(output)
    
    # La linea de abajo por ahora normaliza valores numericos, puesto que esto lo realiza ya el modelo, no es necesario.
    # output = normalize_data(output)
    
    # Etapa final
    return update_db(output)

## Clean data

In [4]:
def extract_name(value):
    ''' extrae de los valores anidados, el valor del campo "name" ''' 
    output = []
    try:
        value = ast.literal_eval(value)
        if isinstance(value, list):
            for dictionary in value:
                output.append(dictionary['name'])
        elif isinstance(value, dict):
            output.append(value['name'])
    finally:
        for i in range(len(output)):
            output[i] = output[i].lower()
        return output
    
def extract_iso(value):
    ''' extrae de los valores anidados, el valor del campo "iso" ''' 
    output = []
    try:
        value = ast.literal_eval(value)
        if isinstance(value, list):
            for dictionary in value:
                for key, value in dictionary.items():
                    if 'iso' in key:
                        output.append(value)
    finally:
        for i in range(len(output)):
            output[i] = output[i].lower()
        return output
    
def parse_zero(v):
    try:
        return float(v)
    except ValueError as e:
        return 0
    
def clean_date(value):
    try:
        year, month, day = value.split('-')
        x, y, z = int(year), int(month), int(day)
        if len(year) == 4:
            if len(month) == 2:
                if len(day) == 2:
                    return value
    except:
        return np.nan

In [5]:
def clean_data(data):
    """En esta etapa: eliminamos columnas y duplicados, extraemos valores e inputamos datos faltantes"""
    
    # Seleccionamos las columnas a utilizar.
    columnas_a_usar = ['belongs_to_collection', 'budget', 'genres', 'overview', 'popularity', 'production_companies',
                       'release_date', 'revenue', 'runtime', 'production_countries', 'spoken_languages', 
                       'title', 'vote_average']
    new_data = data[columnas_a_usar].copy()
    
    # Desanidamos los datos.
    
    # Extraemos el valor del key "name" de las columnas belongs, genres, production_companies
    dstructure_cols = ['belongs_to_collection', 'genres', 'production_companies',
                       'production_countries', 'spoken_languages']
    for c in dstructure_cols:
        new_data[c] = data[c].apply(extract_name)
        if c in ['production_countries', 'spoken_languages']:
            new_data[c + '_iso'] = data[c].apply(extract_iso)
        
    # En budget, popularity, revenue, runtime, vote_average
    cols = ['budget', 'popularity', 'revenue', 'runtime', 'vote_average']
    for c in cols:
        new_data[c] = new_data[c].apply(parse_zero) # Remplazamos por 0 los valores no-numericos
        new_data[c] = new_data[c].apply(lambda v: 0 if v < 0 else v) # Remplazamos por 0 los valores negativos
        new_data[c] = new_data[c].fillna(0) # Remplazamos por 0 los valores faltantes 
    
    # Convertimos a NA cualquier valor en release_date que tenga el formato YYYY-mm-dd
    new_data['release_date'] = new_data['release_date'].apply(clean_date)
    
    # Eliminamos las filas con NA en el campo release_date
    new_data.dropna(subset=['release_date'], axis=0, inplace=True)
    
    # Actualizamos el data type de release_date
    new_data['release_date'] = pd.to_datetime(new_data['release_date'])
    
    # Convertimos a lower case los titulos
    new_data['title'] = new_data['title'].str.lower()
    
    # Eliminamos duplicados (consideramos duplicados aquellas peliculas con el mismo release_date)
    new_data.drop_duplicates(subset=['title', 'release_date'], inplace=True)
    
    return new_data    

<h2> Feature Engineer </h2>

In [6]:
# Creamos la funcion para crear la columna era en object_columns
def create_era(year):
    if isinstance(year, int):
        if year < 1960:
            return 'clasicas'
        elif year < 2000:
            return 'retro'
        elif year < 2021:
            return 'contemporaneas'
    return ''

# Creamos la funcion para crear la columna ranking en object_columns
def create_ranking(vote):
    if vote < 3: # si esta entre (0, 3)
        return 'mala'
    elif vote < 6: # si esta entre (3, 5)
        return 'regular'
    elif vote < 7.5: # si esta entre (5, 7.5)
        return 'buena'
    elif vote < 8.5: # si esta entre (7, 8.5)
        return 'muy buena'
    else: # si es mayor a 8.5
        return 'excelente'
    
# estos criterios son arbitrarios y se pueden probar otros si se quiere.

def limpiar_strings(inlist):
    outlist = []
    for string in inlist:
        # dada una string eliminamos cualquier caracter segun el regex
        x = re.sub(r'[^a-zA-Z0-9\s]', '', string)
        x = ' '.join(x.split())
        outlist.append(x)
    return outlist

def get_keyword(text, nlp, stopwords):
    '''retorna una lista con palabras claves de "text"'''
    if not isinstance(text, str):
        return np.nan
    
    doc = nlp(text)
    
    output = []
    for token in doc.noun_chunks:
        words = token.lemma_.lower().split(' ')
        string = []
        for word in words:
            # eliminamos del token todos los stopwords
            if word in stopwords:
                continue
            string.append(word)
        if len(string) >= 2: # nos quedamos con los "tokens" con mas de 2 palabras
            output.append(' '.join(string))
            
    return limpiar_strings(output)
    
    
def create_doc_vocab(df, text_column=None):
    ''' Crea la columna DOC y retorna Vocab '''
    
    # Si text_column es None, entonces usa todas las columnas no-numericas excepto title.
    if not text_column:
        text_column = df.select_dtypes(exclude=np.number).drop(columns=['title']).columns.tolist()

    # Crea la columna doc, vacia por el momento.
    df['doc'] = ''
    
    # Crea vocab
    vocab = []
    
    
    soup = []
    # Este codigo recorre cada fila, para cada fila hace uso de todas las columnas text_column
    # si una palabra en una columna "C" no se encuentra en vocab, lo agrega.
    # soup es una lista de tokens asociada a cada pelicula/fila
    for i, row in df.iterrows():
        for c in text_column:
            if isinstance(row[c], list):
                for token in row[c]:
                    if token not in vocab:
                        vocab.append(token)
                    soup.append(token)
            elif isinstance(row[c], str):
                if row[c] not in vocab:
                    vocab.append(row[c])
                soup.append(row[c])
        df.loc[i, 'doc'] = ' '.join(soup)
        soup = []
        
    return vocab

In [7]:
def feature_engineer(data):
    # Creamos el objecto nlp
    nlp = spacy.load('en_core_web_sm')
    stopwords = nlp.Defaults.stop_words # stops words basado en "en_core_web_sm"
    
    # Creamos una copia de data
    new_data = data.copy()
    
    # Creamos la columna return y remplazamos los valores nan por 0
    new_data['return'] = new_data['revenue'] / new_data['budget']
    
    # Remplazamos valores con inf o -inf por np.nan
    new_data['return'] = new_data['return'].replace([np.inf, -np.inf], np.nan)
    
    # Remplazamos los valores nan por 0
    new_data['return'] = new_data['return'].fillna(0)
    
    # Creamos un diccionario para traducir los meses
    meses = {'october': 'octubre', 'december': 'diciembre', 'february': 'febrero', 'november': 'noviembre', 
             'september': 'septiembre', 'may': 'mayo', 'april': 'abril', 'august': 'agosto', 'july': 'julio',
             'june': 'junio', 'january': 'enero','march': 'marzo'}

    # Creamos un diccionario para traducir los dias de la semana
    dias = {'monday': 'lunes','friday': 'viernes', 'thursday': 'martes', 'wednesday': 'miercoles',
            'saturday': 'sabado', 'tuesday': 'jueves', 'sunday': 'domingo'}

    # Creamos la columna release-year
    new_data['release_year'] = new_data.release_date.dt.year
    
    # Creamos la columna release_month
    new_data['release_month'] = new_data.release_date.dt.month_name().str.lower().replace(meses)
    
    # Creamos la columna release_day
    new_data['release_day'] = new_data.release_date.dt.day_name().str.lower().replace(dias)
    
    # Creamos la columna era
    new_data['era'] = new_data['release_year'].apply(create_era)
    
    # Creamos la columna ranking
    new_data['ranking'] = new_data['vote_average'].apply(create_ranking)
    
    # Creamos la columna overview_keywords
    new_data['overview_keywords'] = new_data['overview'].apply(lambda v: get_keyword(v, nlp, stopwords))
    
    # Creamos la columna doc
    columnas_para_doc = ['belongs_to_collection', 'genres', 'production_companies', 
                         'production_countries_iso', 'spoken_languages_iso', 'era', 'ranking', 'overview_keywords']
    new_vocab = create_doc_vocab(new_data, columnas_para_doc)
    
    # Actualizamos vocab
    
    # Convertimos la lista a Series object
    updated_vocab = pd.Series(new_vocab)
    
    # Importamos el anterior vocab
    try:
        latest_vocab = pd.read_csv('../data/latest_vocab.csv')
        updated_vocab = pd.concat([latest_vocab, new_vocab]).drop_duplicates()
    except FileNotFoundError:
        print('latest_vocab.csv no existia y fue creado')
    finally:
        # Actualizamos el vocab y lo almacenamos
        updated_vocab.to_csv('../data/latest_vocab.csv')
    
    # Dropeamos las columnas que no vamos a utilizar mas
    columnas_a_usar = ['title', 'doc', 'popularity', 'runtime', 'return', 'budget', 'revenue', 
                       'release_year', 'release_month', 'release_day', 
                       'production_countries', 'production_companies', 'genres', 'spoken_languages']
    new_data = new_data[columnas_a_usar]
    
    return new_data

## Normalizacion

In [8]:
def normalize_data(data):
    
    # Normalizamos los valores numericos.
    
    # Separamos el dataframe entre valores numericos y no-numericos
    numeric_df = data.select_dtypes(include=np.number)
    object_df = data.select_dtypes(exclude=np.number)
    
    # Creamos el objeto de normalizacion
    rscaler = RobustScaler()
    
    # Normalizamos los datos
    numeric_df = rscaler.fit_transform(numeric_df)
    
    return pd.concat([object_df, numeric_df], axis=1)
    

## Actualizar base de datos

In [9]:
def update_db(new_data):
    updated = new_data
    try:
        latest = pd.read_csv('../data/latest_movies.csv')
        updated = pd.concat([latest, new_data], axis=0)
        updated = updated.drop_duplicates(keep='first')
    except FileNotFoundError:
           print('El archivo latest_movies.csv no existia y fue creado')
    finally:
        updated.to_csv('../data/latest.csv', mode='w', header=True, index=False)
        return updated

In [10]:
# Como venian los datos:
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [11]:
# Como quedaron:
start = time.time()
new = pipeline(data)
end = time.time()

latest_vocab.csv no existia y fue creado
El archivo latest_movies.csv no existia y fue creado


In [12]:
print(f'El proceso demoro {(end - start) / 60:.1f} minutos')

El proceso demoro 13.1
